# Trees - Heap

---

Complete binary tree where parent $\geq$ children (max-heap) or $\leq$ children (min-heap).



## Properties:

* **Complete**: All levels full except last (left-filled)
* **Easy to find parents**: `parent(i) = (i-1)//2, left = 2i+1, right = 2i+2`
* Root is always max (max-heap) or min (min-heap)


## Use Cases

* Priority queues (Dijkstra)
* Heap sort in $O(n \log n)$
* $k$-largest/smallest elements

## Interchangeability

Min heap and Max heap are practically the same thing. Say we have a max heap implementation and want to find the minimum of 
$[5, 3, 2, 1, 9]$
via heapifying the array.

Recall that $\min(-a, -b) = -\max(a, b)$, so we can just create a heap on $[-5, -3, -2, -1, -9]$, and the root of the heap is now $-1$ (as its the maximum value). 

In [1]:
from typing import Any, Optional

from theoria.validor import List, TestCase, Validor

In [2]:
class MaxHeap:
    def __init__(self):
        self.heap = []

    # Insert new value (bubble up from end)
    def _insert(self, value: Any) -> None:
        # Add at the end
        self.heap.append(value)

        # Bubble up, halfway each time
        i = len(self.heap) - 1
        while i > 0:
            parent = (i - 1) // 2

            # Stop if parent is bigger (heap property OK)
            if self.heap[parent] >= self.heap[i]:
                break

            # Swap with parent (restore heap property)
            self.heap[parent], self.heap[i] = self.heap[i], self.heap[parent]
            i = parent

    # Extract max value (bubble down from root)
    def _extract(self) -> Optional[Any]:
        if not self.heap:
            return None

        result = self.heap[0]
        if len(self.heap) == 1:
            self.heap.pop()
            return result

        self.heap[0] = self.heap.pop()

        i = 0
        while True:
            # Find children indices
            left = 2 * i + 1
            right = 2 * i + 2

            # Assume current node is largest
            largest = i

            # Check if left child is larger
            if left < len(self.heap) and self.heap[left] > self.heap[largest]:
                largest = left

            # Check if right child is larger
            if right < len(self.heap) and self.heap[right] > self.heap[largest]:
                largest = right

            # If no larger child found, heap property restored
            if largest == i:
                break

            # Swap with largest child
            self.heap[i], self.heap[largest] = self.heap[largest], self.heap[i]
            i = largest

        return result

    def insert(self, values: List[Any]) -> List[Any]:
        for value in values:
            self._insert(value)
        return self.heap

    def extract(self, k: int = 1) -> List[Any]:
        result = []
        for _ in range(k):
            val = self._extract()
            if val is not None:
                result.append(val)
            else:
                break
        return result

    def peek(self) -> Optional[Any]:
        if not self.heap:
            return None
        return self.heap[0]

In [3]:
test_cases = [
    TestCase(
        input_data={"values": [5]},
        expected_output=[5],
        description="Insert single element",
    ),
    TestCase(
        input_data={"values": [5, 3, 8, 1, 2, 7]},
        expected_output=[8, 7, 5, 3, 2, 1],
        description="Basic test case with mixed numbers",
    ),
    TestCase(
        input_data={"values": [10, 20, 15, 30, 40]},
        expected_output=[40, 30, 20, 15, 10],
        description="Insert elements in increasing order",
    ),
    TestCase(
        input_data={"values": [-1, -2, -3, -4, -5]},
        expected_output=[-1, -2, -3, -4, -5],
        description="Use as min-heap via negative numbers",
    ),
]


def insert_and_extract_tests(values: List[Any]) -> List[Any]:
    heap = MaxHeap()
    heap.insert(values)
    result = heap.extract(len(values))
    return result


Validor(insert_and_extract_tests).add_cases(test_cases).run()

[2025-12-05 06:51:51,361] [INFO] All 4 tests passed for insert_and_extract_tests.


## Complexity

| Operation      | Time     | Space | Notes                           |
| -------------- | -------- | ----- | ------------------------------- |
| Insert         | O(log n) | O(1)  | Bubble up height of tree        |
| Extract Max    | O(log n) | O(1)  | Bubble down height of tree      |
| Peek (get max) | O(1)     | O(1)  | Root = max!                     |
| Build Heap     | O(n)     | O(1)  | Bottom-up heapify (not n log n) |
| Search         | O(n)     | O(1)  | No ordering for search          |